In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!mkdir /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCV_Lab/data
!cp /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/data/data.tfrecord /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCV_Lab/data/data.tfrecord

     |████████████████████████████████| 204kB 4.8MB/s 
/content/drive/My Drive/Colab Notebooks/DeepCV_Packages


In [1]:
# install requirements
!cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCV_Lab/ && pip3 install -r requirements.txt

# permanently change dir
%cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCV_Lab

# install waymo dataset utils in utils; https://github.com/waymo-research/waymo-open-dataset/blob/master/tutorial/tutorial.ipynb
!cd utils && rm -rf waymo-od > /dev/null
!cd utils && git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!cd utils/waymo-od && git branch -a
!cd utils/waymo-od && git checkout remotes/origin/r1.0
!pip3 install --upgrade pip
!pip3 install waymo-open-dataset

# directly retrieve data from google
# https://medium.com/@philipplies/transferring-data-from-google-drive-to-google-cloud-storage-using-google-colab-96e088a8c041
'''
from google.colab import auth
auth.authenticate_user()
project_id = 'nifty-depth-246308'
!gcloud config set project {project_id}
!gsutil ls
bucket_name = 'medium_demo_bucket_190710'
!gsutil -m cp -r /content/mnt/My\ Drive/Colab\ Notebooks/DeepCV_Packages/* gs://{bucket_name}/
'''

/content/drive/My Drive/Colab Notebooks/DeepCV_Packages/DeepCV_Lab
Cloning into 'waymo-od'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 700 (delta 34), reused 39 (delta 14), pack-reused 618
Receiving objects: 100% (700/700), 14.17 MiB | 7.25 MiB/s, done.
Resolving deltas: 100% (417/417), done.
Checking out files: 100% (145/145), done.
* master
  remotes/origin/HEAD -> origin/master
  remotes/origin/master
  remotes/origin/r1.0
  remotes/origin/r1.0-tf1.15
  remotes/origin/r1.0-tf2.0
error: Your local changes to the following files would be overwritten by checkout:
	configure.sh
	pip_pkg_scripts/build.sh
Please commit your changes or stash them before you switch branches.
Aborting
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.0.2)


"\nfrom google.colab import auth\nauth.authenticate_user()\nproject_id = 'nifty-depth-246308'\n!gcloud config set project {project_id}\n!gsutil ls\nbucket_name = 'medium_demo_bucket_190710'\n!gsutil -m cp -r /content/mnt/My\\ Drive/Colab\\ Notebooks/DeepCV_Packages/* gs://{bucket_name}/\n"